Using edited code from https://github.com/teyang-lau

In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
## Function for getting postal code, geo coordinates of addresses

def find_postal(lst, filename):
    '''With the block number and street name, get the full address of the hdb flat,
    including the postal code, geogaphical coordinates (lat/long). Can also input postal code instead of full block number
    and street name.'''
    
    for index,add in enumerate(lst):
        # Do not need to change the URL
        url= "https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal="+ add        
        print(index,url)
        
        # Retrieve information from website
        response = requests.get(url)
        try:
            data = json.loads(response.text) 
        except ValueError:
            print('JSONDecodeError')
            pass
    
        temp_df = pd.DataFrame.from_dict(data["results"])
        # The "add" is the address that was used to search in the website
        temp_df["address"] = add
        
        # Create the file with the first row that is read in 
        if index == 0:
            file = temp_df
        else:
            file = file.append(temp_df)
    file.to_csv(filename + '.csv')

In [3]:
## Function for getting closest distance of each location from a list of amenities location

from geopy.distance import geodesic

def find_nearest(house, amenity, radius=2):
    """
    this function finds the nearest locations from the 2nd table from the 1st address
    Both are dataframes with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    It also finds the number of amenities within the given radius (default=2)
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['','',100,0]
        for ind, eachloc in enumerate(amenity.iloc[:,0]):
            amenity_loc = (amenity.iloc[ind,1],amenity.iloc[ind,2]) 
            distance = geodesic(flat_loc,amenity_loc)
            distance = float(str(distance)[:-3]) # convert to float

            if distance <= radius:   # compute number of amenities in 2km radius
                flat_amenity[3] += 1

            if distance < flat_amenity[2]: # find nearest amenity
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
    return results

In [4]:
def dist_from_location(house, location):
    """
    this function finds the distance of a location from the 1st address
    First is a dataframe with a specific format:
        1st column: any string column ie addresses taken from the "find_postal_address.py"
        2nd column: latitude (float)
        3rd column: longitude (float)
    Column name doesn't matter.
    Second is tuple with latitude and longitude of location
    """
    results = {}
    # first column must be address
    for index,flat in enumerate(house.iloc[:,0]):
        
        # 2nd column must be latitude, 3rd column must be longitude
        flat_loc = (house.iloc[index,1],house.iloc[index,2])
        flat_amenity = ['',100]
        distance = geodesic(flat_loc,location)
        distance = float(str(distance)[:-3]) # convert to float
        flat_amenity[0] = flat
        flat_amenity[1] = distance
        results[flat] = flat_amenity
    return results

In [49]:
df = pd.read_csv('HDB_prices_v1.csv')
df = df[['block','street_name']]
df['address'] = df['block'] + ' ' + df['street_name']
all_address = list(df['address'])
unique_address = list(set(all_address))
print('Unique addresses:', len(unique_address))

Unique addresses: 9381


In [54]:
unique_address[0:10]

['302 UBI AVE 1',
 '101A LOR 2 TOA PAYOH',
 '689A WOODLANDS DR 75',
 '658C JURONG WEST ST 65',
 '650A JURONG WEST ST 61',
 '430 ANG MO KIO AVE 3',
 '356 BT BATOK ST 31',
 '539 BT PANJANG RING RD',
 '315 UBI AVE 1',
 '706 CHOA CHU KANG ST 53']

In [51]:
find_postal(unique_address, 'flat_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=302 UBI AVE 1
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=101A LOR 2 TOA PAYOH
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=689A WOODLANDS DR 75
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=658C JURONG WEST ST 65
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=650A JURONG WEST ST 61
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=430 ANG MO KIO AVE 3
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=356 BT BATOK ST 31
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=539 BT PANJANG RING RD
8 https://developers.onemap.sg/commonapi/search?ret

74 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=451 FAJAR RD
75 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=241 BISHAN ST 22
76 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=873 YISHUN ST 81
77 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=815 TAMPINES AVE 4
78 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=207D COMPASSVALE LANE
79 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=615 BT PANJANG RING RD
80 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=270C PUNGGOL FIELD
81 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=168 HOUGANG AVE 1
82 https://developers.onemap.sg/commonapi/search?returnGeom=

144 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=303B ANCHORVALE LINK
145 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=684D CHOA CHU KANG CRES
146 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=201B TAMPINES ST 21
147 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=26D JLN MEMBINA
148 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=53 TANGLIN HALT RD
149 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=404 JURONG WEST ST 42
150 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=739 TAMPINES ST 72
151 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=730 JURONG WEST ST 72
152 https://developers.onemap.sg/comm

214 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=579 WOODLANDS DR 16
215 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=20 BEDOK STH RD
216 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=227 YISHUN ST 21
217 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=110 LOR 1 TOA PAYOH
218 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=604B PUNGGOL RD
219 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=274 CHOA CHU KANG AVE 2
220 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=1G CANTONMENT RD
221 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=723 CLEMENTI WEST ST 2
222 https://developers.onemap.sg/commonapi/sear

285 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=316 JURONG EAST ST 32
286 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=979A BUANGKOK CRES
287 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=273C COMPASSVALE LINK
288 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=976 JURONG WEST ST 93
289 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=485 JURONG WEST AVE 1
290 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=610 BEDOK RESERVOIR RD
291 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=97 GEYLANG BAHRU
292 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=327 SEMBAWANG CRES
293 https://developers.onemap.sg/c

...

9067 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=690E WOODLANDS DR 75
9068 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=637 VEERASAMY RD
9069 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=321 CLEMENTI AVE 5
9070 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=105A EDGEFIELD PLAINS
9071 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=462 HOUGANG AVE 10
9072 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=15 ST. GEORGE'S RD
9073 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=233 YISHUN ST 21
9074 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=17 TELOK BLANGAH CRES
9075 https://developers.onemap.sg/co

9138 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=263 JURONG EAST ST 24
9139 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=130 PASIR RIS ST 11
9140 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=421 WOODLANDS ST 41
9141 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=103 DEPOT RD
9142 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=271A PUNGGOL WALK
9143 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=698D HOUGANG ST 52
9144 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=140 YISHUN RING RD
9145 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=702 WOODLANDS DR 40
9146 https://developers.onemap.sg/commona

9207 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=717 WOODLANDS DR 70
9208 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=433B SENGKANG WEST WAY
9209 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=642 PASIR RIS DR 10
9210 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=120 POTONG PASIR AVE 1
9211 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=118 PENDING RD
9212 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=309 SERANGOON AVE 2
9213 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=58 LOR 4 TOA PAYOH
9214 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=610 JURONG WEST ST 65
9215 https://developers.onemap

9277 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=25 TANGLIN HALT RD
9278 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=771 CHOA CHU KANG ST 54
9279 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=112 YISHUN RING RD
9280 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=466 ANG MO KIO AVE 10
9281 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=788 CHOA CHU KANG NTH 6
9282 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=225 LOR 8 TOA PAYOH
9283 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=513 JURONG WEST ST 52
9284 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=226 PASIR RIS ST 21
9285 https://developer

9347 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=752 YISHUN ST 72
9348 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=122 PASIR RIS ST 11
9349 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=109 JLN BT MERAH
9350 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=610 WOODLANDS AVE 4
9351 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=51 JLN BAHAGIA
9352 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=737 TAMPINES ST 72
9353 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=309 SHUNFU RD
9354 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=620 BT BATOK CTRL
9355 https://developers.onemap.sg/commonapi/search?r

In [52]:
flat_coord = pd.read_csv('data/flat_coordinates.csv')
flat_coord = flat_coord[['address','LATITUDE','LONGITUDE']]
flat_coord.head()

,address,LATITUDE,LONGITUDE
0,302 UBI AVE 1,1.330026,103.900995
1,302 UBI AVE 1,1.330511,103.901088
2,101A LOR 2 TOA PAYOH,1.339741,103.847264
3,689A WOODLANDS DR 75,1.444386,103.807925
4,658C JURONG WEST ST 65,1.336853,103.700477


In [55]:
flat_coord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11590 entries, 0 to 11589
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   address    11590 non-null  object 
 1   LATITUDE   11590 non-null  float64
 2   LONGITUDE  11590 non-null  float64
dtypes: float64(2), object(1)
memory usage: 226.4+ KB


### Supermarkets

In [56]:
supermarket = pd.read_csv('data/list-of-supermarket-licences.csv')
supermarket.head()

,licence_num,licensee_name,building_name,block_house_num,level_num,unit_num,street_name,postal_code
0,S02185J000,COLD STORAGE SINGAPORE (1983) PTE LTD,na,982,1,1,BUANGKOK CRESCENT,530982
1,S97166P000,COLD STORAGE SINGAPORE (1983) PTE LTD,HOUGANG GREEN SHOPPING MALL,21,1,52,HOUGANG STREET 51,538719
2,CE15B61K000,U STARS SUPERMARKET PTE. LTD.,na,330,1,1,ANCHORVALE STREET,540330
3,S02029J000,NTUC Fairprice Co-operative Ltd,HOUGANG POINT,1,2,1,HOUGANG STREET 91,538692
4,CE04561V000,SHENG SIONG SUPERMARKET PTE LTD,na,19,1,42401,SERANGOON NORTH AVENUE 5,554913


In [57]:
supermarket_address = list(supermarket['postal_code'])
unique_supermarket_address = list(set(supermarket_address))
print('Unique addresses:', len(unique_supermarket_address))

Unique addresses: 445


In [58]:
find_postal(unique_supermarket_address, 'data/supermarket_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=389290
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=389344
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=460509
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=389797
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=670628
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=400306
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=209586
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=420002
8 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=649698
9 https://developers.onemap.sg/commonapi/search?returnG

79 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=469979
80 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=310192
81 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=797653
82 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=289236
83 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=757388
84 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=088488
85 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=120502
86 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=679947
87 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=639798
88 https://developers.onemap.sg/commonapi/sear

158 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=141088
159 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=824622
160 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=330071
161 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=636947
162 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=510259
163 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=408600
164 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=550266
165 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=471739
166 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=659958
167 https://developers.onemap.sg/comm

237 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=380011
238 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=680475
239 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=560421
240 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=610063
241 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=120108
242 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=448912
243 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=799473
244 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=540279
245 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=718813
246 https://developers.onemap.sg/comm

316 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=050004
317 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=589831
318 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=689812
319 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=179103
320 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=737723
321 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=588177
322 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=169038
323 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=730900
324 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=510440
325 https://developers.onemap.sg/comm

393 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=636937
394 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=600061
395 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=101078
396 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=610399
397 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=540266
398 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=238801
399 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=637285
400 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=819663
401 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=415814
402 https://developers.onemap.sg/comm

In [40]:
supermarket_coord = pd.read_csv('data/supermarket_coordinates.csv')
supermarket_coord.drop_duplicates(subset=['address'], inplace=True)
supermarket_coord = supermarket_coord[['SEARCHVAL','LATITUDE','LONGITUDE']]
supermarket_coord.head()

,SEARCHVAL,LATITUDE,LONGITUDE
0,230 GEYLANG ROAD SINGAPORE 389290,1.312088,103.877389
1,BCH@301,1.312724,103.878705
2,509 BEDOK NORTH STREET 3 SINGAPORE 460509,1.333175,103.931265
7,5 ALJUNIED ROAD SINGAPORE 389797,1.313576,103.881878
8,SENJA CENTRE,1.385096,103.760218


In [59]:
nearest_supermarket = find_nearest(flat_coord, supermarket_coord)
flat_supermarket = pd.DataFrame.from_dict(nearest_supermarket).T
flat_supermarket = flat_supermarket.rename(columns={0: 'flat', 1: 'supermarket', 2: 'supermarket_dist', 3: 'num_supermarket_2km'}).reset_index().drop(['index','supermarket'], axis=1)
flat_supermarket.head()

,flat,supermarket_dist,num_supermarket_2km
0,302 UBI AVE 1,0.117485,20
1,101A LOR 2 TOA PAYOH,0.749608,15
2,689A WOODLANDS DR 75,0.274268,19
3,658C JURONG WEST ST 65,0.557519,8
4,650A JURONG WEST ST 61,0.395229,9


### Schools

In [60]:
schools = pd.read_csv('data/school_names.csv', encoding='cp1252')
schools.head()

,school,area
0,Admiralty Primary School,Woodlands
1,Ahmad Ibrahim Primary School,Yishun
2,Ai Tong School,Bishan
3,Alexandra Primary School,Bukit Merah
4,Anchor Green Primary School,Sengkang


In [62]:
school_name = list(schools['school'])
unique_school_name = list(set(school_name))

print('Unique addresses:', len(unique_school_name))

Unique addresses: 191


In [63]:
find_postal(unique_school_name, 'data/school_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Lakeside Primary School
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=New Town Primary School
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Guangyang Primary School
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Innova Primary School
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Princess Elizabeth Primary School
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Dazhong Primary School
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Punggol View Primary School
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Frontier Primary School
8 https://de

66 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Woodlands Ring Primary School
67 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Zhangde Primary School
68 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Huamin Primary School
69 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Fernvale Primary School
70 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Angsana Primary School
71 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Catholic High School (Primary)
72 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Damai Primary School
73 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Kheng Cheng School
74 https://deve

132 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Haig Girls' School
133 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Fuhua Primary School
134 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Unity Primary School
135 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Woodlands Primary School
136 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Punggol Green Primary School
137 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Radin Mas Primary School
138 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Opera Estate Primary School
139 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=CHIJ Primary (Toa Payoh)
140 htt

In [64]:
school_coord = pd.read_csv('data/school_coordinates.csv')
school_coord = school_coord[['address','LATITUDE','LONGITUDE']]
school_coord.head()

,address,LATITUDE,LONGITUDE
0,Lakeside Primary School,1.338376,103.718097
1,Lakeside Primary School,1.338376,103.718051
2,New Town Primary School,1.299811,103.799965
3,Guangyang Primary School,1.346043,103.849285
4,Innova Primary School,1.429185,103.790557


In [65]:
nearest_school = find_nearest(flat_coord, school_coord)
flat_school = pd.DataFrame.from_dict(nearest_school).T
flat_school = flat_school.rename(columns={0: 'flat', 1: 'school', 2: 'school_dist', 3: 'num_school_2km'}).reset_index().drop('index', axis=1)
flat_school.head()

,flat,school,school_dist,num_school_2km
0,302 UBI AVE 1,Maha Bodhi School,0.21492,5
1,101A LOR 2 TOA PAYOH,Kheng Cheng School,0.263901,13
2,689A WOODLANDS DR 75,Greenwood Primary School,0.5984,18
3,658C JURONG WEST ST 65,Frontier Primary School,0.0914216,16
4,650A JURONG WEST ST 61,Jurong Primary School,0.259749,14


#### Hawker and Markets

In [66]:
hawker = pd.read_csv('data/list-of-government-markets-hawker-centres.csv')
hawker.head(10)

,name_of_centre,location_of_centre,type_of_centre,owner,no_of_stalls,no_of_cooked_food_stalls,no_of_mkt_produce_stalls
0,Adam Road Food Centre,"2, Adam Road, S(289876)",HC,Government,32,32,0
1,Amoy Street Food Centre,"National Development Building, Annex B, Telok ...",HC,Government,135,134,1
2,Bedok Food Centre,"1, Bedok Road, S(469572)",HC,Government,32,32,0
3,Beo Crescent Market,"38A, Beo Crescent, S(169982)",MHC,Government,94,32,62
4,Berseh Food Centre,"166, Jalan Besar, S(208877)",HC,Government,66,66,0
5,Bukit Timah Market,"51, Upper Bukit Timah Road, S(588215)",MHC,Government,179,84,95
6,Chomp Chomp Food Centre,"20, Kensington Park Road, S(557269)",HC,Government,36,36,0
7,Commonwealth Crescent Market,"31, Commonwealth Crescent, S(149644)",MHC,Government,102,39,63
8,Dunman Food Centre,"271, Onan Road, S(424768)",HC,Government,30,30,0
9,East Coast Lagoon Food Village,"1220, East Coast Parkway, S(468960)",HC,Government,63,63,0


In [67]:
hawker_name = list(hawker['name_of_centre'])
unique_hawker_name = list(set(hawker_name))

print('Unique addresses:', len(unique_hawker_name))

Unique addresses: 107


In [68]:
find_postal(unique_hawker_name, 'data/hawker_coordinates')

0 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Kovan Market & Food Centre
1 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 11 Telok Blangah Crescent
2 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 75 Toa Payoh Lorong 5
3 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 4A Eunos Crescent
4 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 503 West Coast Drive
5 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Maxwell Food Centre
6 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 90 Whampoa Drive
7 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Berseh Food Centre
8 https://developers.onema

68 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 347 Jurong East Ave 1
69 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 1 Jalan Kukoh
70 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 527 Ang Mo Kio Ave 10
71 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 16 Bedok South Road
72 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Taman Jurong Market & Food Centre
73 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 105 Hougang Ave 1
74 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Blk 502 West  Coast Drive
75 https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&pageNum=1&searchVal=Chinatown Market
76 https://deve

In [69]:
hawker_coord = pd.read_csv('Data/hawker_coordinates.csv')
hawker_coord = hawker_coord[['address','LATITUDE','LONGITUDE']]
hawker_coord.head()

,address,LATITUDE,LONGITUDE
0,Kovan Market & Food Centre,1.359097,103.886062
1,Blk 11 Telok Blangah Crescent,1.277367,103.818642
2,Blk 75 Toa Payoh Lorong 5,1.335989,103.852993
3,Blk 4A Eunos Crescent,1.320341,103.904331
4,Maxwell Food Centre,1.280538,103.844475


In [70]:
nearest_hawker = find_nearest(flat_coord, hawker_coord)
flat_hawker = pd.DataFrame.from_dict(nearest_hawker).T
flat_hawker = flat_hawker.rename(columns={0: 'flat', 1: 'hawker', 2: 'hawker_dist', 3: 'num_hawker_2km'}).reset_index().drop('index', axis=1)
flat_hawker.head()

,flat,hawker,hawker_dist,num_hawker_2km
0,302 UBI AVE 1,Blk 4A Eunos Crescent,1.18113,9
1,101A LOR 2 TOA PAYOH,Blk 93 Toa Payoh Lorong 4,0.287251,6
2,689A WOODLANDS DR 75,Chong Pang Market & Food Centre,2.64796,0
3,658C JURONG WEST ST 65,Taman Jurong Market & Food Centre,2.35516,0
4,650A JURONG WEST ST 61,Blk 505 Jurong West Street 52,2.72031,0


### MRT

In [72]:
mrt_coord = pd.read_csv('data/MRT_coordinates.csv')
mrt_coord = mrt_coord[['STN_NAME','Latitude','Longitude']]
mrt_coord.head()

,STN_NAME,Latitude,Longitude
0,ADMIRALTY MRT STATION,1.440585,103.800998
1,ALJUNIED MRT STATION,1.316433,103.882893
2,ANG MO KIO MRT STATION,1.369933,103.849553
3,BAKAU LRT STATION,1.388093,103.905418
4,BANGKIT LRT STATION,1.380018,103.772667


In [73]:
nearest_mrt = find_nearest(flat_coord, mrt_coord)
flat_mrt = pd.DataFrame.from_dict(nearest_mrt).T
flat_mrt = flat_mrt.rename(columns={0: 'flat', 1: 'mrt', 2: 'mrt_dist', 3: 'num_mrt_2km'}).reset_index().drop('index', axis=1)
flat_mrt.head()

,flat,mrt,mrt_dist,num_mrt_2km
0,302 UBI AVE 1,UBI MRT STATION,0.215147,10
1,101A LOR 2 TOA PAYOH,BRADDELL MRT STATION,0.096223,7
2,689A WOODLANDS DR 75,ADMIRALTY MRT STATION,0.878036,2
3,658C JURONG WEST ST 65,PIONEER MRT STATION,0.356433,2
4,650A JURONG WEST ST 61,PIONEER MRT STATION,0.069539,2


In [74]:
flat_coord = pd.read_csv('data/flat_coordinates.csv')
flat_coord = flat_coord[['address','LATITUDE','LONGITUDE']]
flat_coord.head()

,address,LATITUDE,LONGITUDE
0,302 UBI AVE 1,1.330026,103.900995
1,302 UBI AVE 1,1.330511,103.901088
2,101A LOR 2 TOA PAYOH,1.339741,103.847264
3,689A WOODLANDS DR 75,1.444386,103.807925
4,658C JURONG WEST ST 65,1.336853,103.700477


### Get Distance to Central 
We will use Raffles Place MRT as central.

In [81]:
flat_coord = pd.read_csv('data/flat_coordinates.csv')
flat_coord = flat_coord[['address','LATITUDE','LONGITUDE']]
dist_central = dist_from_location(flat_coord, (1.2841251, 103.8514384))
dist_central = pd.DataFrame.from_dict(dist_central).T
dist_central = dist_central.rename(columns={0: 'flat', 1: 'dist_central'}).reset_index().drop(['index'], axis=1)
dist_central.head()

,flat,dist_central
0,302 UBI AVE 1,7.5392
1,101A LOR 2 TOA PAYOH,6.16727
2,689A WOODLANDS DR 75,18.3706
3,658C JURONG WEST ST 65,17.7835
4,650A JURONG WEST ST 61,18.2014


### Merge all

In [80]:
## Merge all

flat_amenities = flat_school.merge(flat_hawker, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_mrt, on='flat', how='outer')
flat_amenities = flat_amenities.merge(flat_supermarket, on='flat', how='outer')
flat_amenities = flat_amenities.merge(dist_central, on='flat', how='outer')
flat_amenities.head()

,flat,school,school_dist,num_school_2km,hawker,hawker_dist,num_hawker_2km,mrt,mrt_dist,num_mrt_2km,supermarket_dist,num_supermarket_2km,dist_central
0,302 UBI AVE 1,Maha Bodhi School,0.21492,5,Blk 4A Eunos Crescent,1.18113,9,UBI MRT STATION,0.215147,10,0.117485,20,7.5392
1,101A LOR 2 TOA PAYOH,Kheng Cheng School,0.263901,13,Blk 93 Toa Payoh Lorong 4,0.287251,6,BRADDELL MRT STATION,0.096223,7,0.749608,15,6.16727
2,689A WOODLANDS DR 75,Greenwood Primary School,0.5984,18,Chong Pang Market & Food Centre,2.64796,0,ADMIRALTY MRT STATION,0.878036,2,0.274268,19,18.3706
3,658C JURONG WEST ST 65,Frontier Primary School,0.0914216,16,Taman Jurong Market & Food Centre,2.35516,0,PIONEER MRT STATION,0.356433,2,0.557519,8,17.7835
4,650A JURONG WEST ST 61,Jurong Primary School,0.259749,14,Blk 505 Jurong West Street 52,2.72031,0,PIONEER MRT STATION,0.069539,2,0.395229,9,18.2014


In [82]:
flat_amenities.to_csv('data/flat_amenities.csv', index=False)